In [ ]:
!pip install nbformat
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
from scipy.ndimage import gaussian_filter1d
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import pickle
import warnings
warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
TRAIN_YEARS = 10  # Use 10 years data to train
FORECAST_DAYS = 91       # USER INPUT
CV_HORIZON = '91 days'
CV_PERIOD = '182 days'
CV_INITIAL = '3285 days'

# SMOOTHING CONFIGURATION
SMOOTHING_STRENGTH = 0.7
USE_HYBRID = True # Use Prophet + ARIMA hybrid approach
USE_INTERACTIVE_CHARTS = True  

# MODEL SAVING CONFIGURATION
SAVE_MODELS = True
MODEL_FOLDER = 'SaveModels'  # Folder to save models

# Stock files list
df_files = [
    'dataset/AMZN_stock_data.csv', 'dataset/AAPL_stock_data.csv', 'dataset/MMM_stock_data.csv', 'dataset/BA_stock_data.csv',
    'dataset/AXP_stock_data.csv', 'dataset/CAT_stock_data.csv', 'dataset/CRM_stock_data.csv', 'dataset/CSCO_stock_data.csv',
    'dataset/CVX_stock_data.csv', 'dataset/DIS_stock_data.csv', 'dataset/GS_stock_data.csv', 'dataset/HD_stock_data.csv',
    'dataset/HON_stock_data.csv', 'dataset/IBM_stock_data.csv', 'dataset/JNJ_stock_data.csv', 'dataset/JPM_stock_data.csv',
    'dataset/MCD_stock_data.csv', 'dataset/MSFT_stock_data.csv', 'dataset/NVDA_stock_data.csv', 'dataset/PG_stock_data.csv',
    'dataset/SHW_stock_data.csv', 'dataset/TRV_stock_data.csv', 'dataset/UNH_stock_data.csv', 'dataset/V_stock_data.csv'
]

def add_technical_features(df):
    # Exponential moving averages
    df['EMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EMA_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    
    # Smooth volatility
    df['Volatility_21'] = df['Close'].rolling(window=21, min_periods=1).std()
    df['Volatility_21'] = df['Volatility_21'].ewm(span=5, adjust=False).mean()
    
    # Smooth momentum
    df['Momentum_14'] = df['Close'].pct_change(periods=14).fillna(0)
    df['Momentum_14'] = df['Momentum_14'].ewm(span=5, adjust=False).mean()
    
    # Volume indicators
    df['Volume_EMA_21'] = df['Volume'].ewm(span=21, adjust=False).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_EMA_21']
    df['Volume_Ratio'] = df['Volume_Ratio'].ewm(span=5, adjust=False).mean()
    
    # Fill any remaining NaN values
    df = df.fillna(method='bfill').fillna(method='ffill')
    
    return df

def smooth_forecast(forecast_series, strength=0.7):
    sigma = strength * 5  # Sigma controls smoothing strength
    smoothed = gaussian_filter1d(forecast_series, sigma=sigma)
    return smoothed

def fit_arima_trend(data, order=(2, 1, 2)):
    try:
        model = ARIMA(data, order=order, enforce_stationarity=False, enforce_invertibility=False)
        fitted_model = model.fit()
        return fitted_model
    except:
        # Fallback to simpler model
        model = ARIMA(data, order=(1, 1, 1), enforce_stationarity=False)
        return model.fit()

def create_interactive_chart(stock_name, last_90_days, future_forecast, accuracy, 
                             forecast_days, train_start_price):
    # Prepare data for historical (last 90 days)
    hist_dates = last_90_days['ds']
    hist_prices = last_90_days['y']
    hist_changes = hist_prices - hist_prices.iloc[0]
    hist_pct_changes = (hist_changes / hist_prices.iloc[0]) * 100
    
    # Prepare data for forecast
    forecast_dates = future_forecast['ds']
    forecast_prices = future_forecast['yhat']
    forecast_lower = future_forecast['yhat_lower']
    forecast_upper = future_forecast['yhat_upper']
    
    # Calculate changes from last historical price
    last_hist_price = hist_prices.iloc[-1]
    forecast_changes = forecast_prices - last_hist_price
    forecast_pct_changes = (forecast_changes / last_hist_price) * 100
    
    # Create figure with subplots
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.7, 0.3],
        vertical_spacing=0.1,
        subplot_titles=(
            f'{stock_name} - Interactive {forecast_days}-Day Forecast | Accuracy: {accuracy:.1f}%',
            'Daily Price Change (%)'
        ),
        specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
    )
    
    # Historical prices (last 90 days)
    fig.add_trace(
        go.Scatter(
            x=hist_dates,
            y=hist_prices,
            name='Historical Price',
            mode='lines',
            line=dict(color='#2E86AB', width=3),
            hovertemplate='<b>Historical</b><br>' +
                         'Date: %{x|%d-%m-%Y}<br>' +
                         'Price: $%{y:.2f}<br>' +
                         'Change: $%{customdata[0]:.2f}<br>' +
                         'Change %: %{customdata[1]:.2f}%<br>' +
                         '<extra></extra>',
            customdata=np.column_stack((hist_changes, hist_pct_changes))
        ),
        row=1, col=1
    )
    
    # Confidence interval
    fig.add_trace(
        go.Scatter(
            x=forecast_dates,
            y=forecast_upper,
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=forecast_dates,
            y=forecast_lower,
            mode='lines',
            line=dict(width=0),
            fillcolor='rgba(162, 59, 114, 0.2)',
            fill='tonexty',
            name='95% Confidence',
            showlegend=True,
            hoverinfo='skip'
        ),
        row=1, col=1
    )
    
    # Forecast line
    fig.add_trace(
        go.Scatter(
            x=forecast_dates,
            y=forecast_prices,
            name='Forecast',
            mode='lines',
            line=dict(color='#A23B72', width=4),
            hovertemplate='<b>Forecast</b><br>' +
                         'Date: %{x|%d-%m-%Y}<br>' +
                         'Price: $%{y:.2f}<br>' +
                         'Change: $%{customdata[0]:.2f}<br>' +
                         'Change %: %{customdata[1]:+.2f}%<br>' +
                         '<extra></extra>',
            customdata=np.column_stack((forecast_changes, forecast_pct_changes))
        ),
        row=1, col=1
    )
    
    # Start and end markers
    fig.add_trace(
        go.Scatter(
            x=[forecast_dates.iloc[0]],
            y=[forecast_prices.iloc[0]],
            mode='markers',
            marker=dict(size=12, color='#06D6A0', symbol='circle', 
                       line=dict(color='white', width=2)),
            name='Forecast Start',
            hovertemplate='<b>Forecast Start</b><br>' +
                         'Date: %{x|%d-%m-%Y}<br>' +
                         'Price: $%{y:.2f}<br>' +
                         '<extra></extra>'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=[forecast_dates.iloc[-1]],
            y=[forecast_prices.iloc[-1]],
            mode='markers',
            marker=dict(size=12, color='#EF476F', symbol='circle',
                       line=dict(color='white', width=2)),
            name='Forecast End',
            hovertemplate='<b>Forecast End</b><br>' +
                         'Date: %{x|%d-%m-%Y}<br>' +
                         'Price: $%{y:.2f}<br>' +
                         'Total Change: %{customdata[0]:+.2f}%<br>' +
                         '<extra></extra>',
            customdata=[[forecast_pct_changes.iloc[-1]]]
        ),
        row=1, col=1
    )
    
    # Historical daily changes
    hist_daily_changes = last_90_days['y'].pct_change() * 100
    colors_hist = ['#06D6A0' if x >= 0 else '#EF476F' for x in hist_daily_changes]
    
    fig.add_trace(
        go.Bar(
            x=hist_dates,
            y=hist_daily_changes,
            name='Historical Daily %',
            marker_color=colors_hist,
            hovertemplate='Date: %{x|%d-%m-%Y}<br>' +
                         'Daily Change: %{y:.2f}%<br>' +
                         '<extra></extra>',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Forecast daily changes
    forecast_daily_changes = forecast_prices.pct_change() * 100
    colors_forecast = ['#06D6A0' if x >= 0 else '#EF476F' for x in forecast_daily_changes]
    
    fig.add_trace(
        go.Bar(
            x=forecast_dates,
            y=forecast_daily_changes,
            name='Forecast Daily %',
            marker_color=colors_forecast,
            hovertemplate='Date: %{x|%d-%m-%Y}<br>' +
                         'Daily Change: %{y:.2f}%<br>' +
                         '<extra></extra>',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Layout
    fig.update_xaxes(title_text="Date", row=1, col=1, showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')
    fig.update_xaxes(title_text="Date", row=2, col=1, showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')
    fig.update_yaxes(title_text="Stock Price ($)", row=1, col=1, showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')
    fig.update_yaxes(title_text="Daily Change (%)", row=2, col=1, showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')
    
    fig.update_layout(
        height=900,
        hovermode='x unified',
        template='plotly_white',
        font=dict(family="Arial, sans-serif", size=12),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        margin=dict(l=80, r=80, t=100, b=80)
    )
    
    return fig

def create_static_chart(stock_name, last_90_days, future_forecast, accuracy, forecast_days):
    plt.figure(figsize=(16, 7))
    
    # Plot last 90 days of historical
    plt.plot(last_90_days['ds'], last_90_days['y'], 
            label='Recent Historical (Last 90 Days)', 
            color='#2E86AB', linewidth=2.5, alpha=0.9)
    
    # Plot the SMOOTH forecast
    plt.plot(future_forecast['ds'], future_forecast['yhat'], 
            label=f'Smooth Forecast ({forecast_days} Days)', 
            color='#A23B72', linewidth=3.5, linestyle='-', alpha=0.95)
    
    # Smooth confidence interval
    plt.fill_between(future_forecast['ds'], 
                    future_forecast['yhat_lower'], 
                    future_forecast['yhat_upper'],
                    color='#A23B72', alpha=0.15, 
                    label='95% Confidence Interval')
    
    # Markers
    plt.scatter(future_forecast['ds'].iloc[0], future_forecast['yhat'].iloc[0],
               color='#06D6A0', s=150, zorder=5, label='Forecast Start', 
               edgecolors='white', linewidths=2)
    plt.scatter(future_forecast['ds'].iloc[-1], future_forecast['yhat'].iloc[-1],
               color='#EF476F', s=150, zorder=5, label='Forecast End',
               edgecolors='white', linewidths=2)
    
    # Vertical line at forecast start
    forecast_start = last_90_days['ds'].iloc[-1]
    plt.axvline(x=forecast_start, color='#06D6A0', 
               linestyle='--', linewidth=2.5, alpha=0.6)
    
    # Price annotations
    start_price = future_forecast['yhat'].iloc[0]
    end_price = future_forecast['yhat'].iloc[-1]
    price_change = end_price - start_price
    pct_change = (price_change / start_price) * 100
    
    # Annotation box
    plt.text(future_forecast['ds'].iloc[-1], end_price, 
            f'  ${end_price:.2f}\n  ({pct_change:+.1f}%)',
            fontsize=12, fontweight='bold', color='#A23B72',
            bbox=dict(boxstyle='round,pad=0.7', facecolor='white', 
                     edgecolor='#A23B72', linewidth=2, alpha=0.9))
    
    plt.title(f"{stock_name} - Smooth {forecast_days}-Day Forecast | Accuracy: {accuracy:.1f}%", 
             fontsize=17, fontweight='bold', pad=20)
    plt.xlabel('Date', fontsize=13, fontweight='bold')
    plt.ylabel('Stock Price ($)', fontsize=13, fontweight='bold')
    plt.legend(loc='best', fontsize=11, framealpha=0.95, edgecolor='gray', fancybox=True)
    plt.grid(alpha=0.25, linestyle='--', linewidth=0.8)
    plt.tight_layout()
    plt.show()

def save_model(model, stock_name, folder='SaveModels'):
    try:
        # Ensure folder exists
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f" Created folder: {folder}")
        
        # Create filename
        filename = f"{stock_name}_model.pkl"
        filepath = os.path.join(folder, filename)
        
        # Save model
        with open(filepath, 'wb') as f:
            pickle.dump(model, f)
        
        print(f" Model saved: {filepath}")
        return filepath
        
    except Exception as e:
        print(f" Error saving model for {stock_name}: {e}")
        return None

def load_model(stock_name, folder='SaveModels'):
    try:
        filename = f"{stock_name}_model.pkl"
        filepath = os.path.join(folder, filename)
        
        if not os.path.exists(filepath):
            print(f" Model file not found: {filepath}")
            return None
        
        with open(filepath, 'rb') as f:
            model = pickle.load(f)
        
        print(f" Model loaded: {filepath}")
        return model
        
    except Exception as e:
        print(f" Error loading model for {stock_name}: {e}")
        return None
   
def optimize_prophet_model(train_df, stock_name, custom_params=None):
    if custom_params:
        # Use tuned parameters
        print(f" Using custom-tuned parameters for {stock_name}")
        model = Prophet(
            changepoint_prior_scale=custom_params['changepoint_prior_scale'],
            seasonality_prior_scale=custom_params['seasonality_prior_scale'],
            seasonality_mode=custom_params['seasonality_mode'],
            changepoint_range=custom_params['changepoint_range'],
            daily_seasonality=False,
            weekly_seasonality=False,
            yearly_seasonality=True,
        )
        
        model.add_country_holidays(country_name='US')
        model.add_seasonality(name='monthly', period=30.5, fourier_order=3)
        model.add_seasonality(name='quarterly', period=91.25, 
                            fourier_order=custom_params.get('fourier_quarterly', 4))
    else:
        # Use volatility-based defaults
        volatility = train_df['y'].pct_change().std()
        
        if volatility > 0.03:  # High volatility stocks
            model = Prophet(
                changepoint_prior_scale=0.03,
                seasonality_prior_scale=3,
                seasonality_mode='multiplicative',
                changepoint_range=0.85,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=True,
            )
        elif volatility > 0.015:  # Medium volatility
            model = Prophet(
                changepoint_prior_scale=0.02,
                seasonality_prior_scale=5,
                seasonality_mode='additive',
                changepoint_range=0.82,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=True,
                interval_width=0.95
            )
        else:  # Low volatility
            model = Prophet(
                changepoint_prior_scale=0.01,
                seasonality_prior_scale=8,
                seasonality_mode='additive',
                changepoint_range=0.80,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=True,
            )
        
        model.add_country_holidays(country_name='US')
        model.add_seasonality(name='monthly', period=30.5, fourier_order=3)
        model.add_seasonality(name='quarterly', period=91.25, fourier_order=4)
    
    # Add smooth regressors
    regressor_cols = ['EMA_7', 'EMA_21', 'EMA_50', 'Volatility_21', 
                     'Momentum_14', 'Volume_Ratio']
    
    for col in regressor_cols:
        if col in train_df.columns:
            model.add_regressor(col, standardize='auto')
    
    return model

metrics_summary = []

for stock_file in df_files:
    print("=" * 50)
    print(f" Training smooth forecast for: {stock_file}")
    print("=" * 50)

    try:
        # --- Load data ---
        df = pd.read_csv(stock_file, sep='|')
        date_col = 'Date'
        
        df[date_col] = pd.to_datetime(df[date_col])
        df = df.sort_values(by=date_col).reset_index(drop=True)
        
        # --- Keep only last 10 years ---
        last_date = df[date_col].max()
        cutoff_date = last_date - pd.DateOffset(years=TRAIN_YEARS)
        df = df[df[date_col] >= cutoff_date].reset_index(drop=True)
        
        print(f"Using data from {df[date_col].min().date()} to {df[date_col].max().date()}")
        print(f"Total trading days: {len(df)}")
        
        # --- Add smooth technical features ---
        df = add_technical_features(df)
        
        stock_name = os.path.basename(stock_file).split('_')[0]
        
        # Create training dataframe
        train = pd.DataFrame({'ds': df[date_col], 'y': df['Close']})
        
        regressor_cols = ['EMA_7', 'EMA_21', 'EMA_50', 'Volatility_21', 
                         'Momentum_14', 'Volume_Ratio']
        
        for col in regressor_cols:
            if col in df.columns:
                train[col] = df[col]
        
        # --- Train Prophet model ---
        model = optimize_prophet_model(train, stock_name)
        model.fit(train)
        
        # --- Save Models
        if SAVE_MODELS:
            save_model(model, stock_name, folder=MODEL_FOLDER)
        # --- Create future dataframe ---
        future = model.make_future_dataframe(periods=FORECAST_DAYS)
        
        # Extrapolate regressors smoothly
        for col in regressor_cols:
            if col in train.columns:
                # Use exponential weighted mean of last 30 values
                last_values = train[col].tail(30).values
                trend = np.mean(np.diff(last_values))
                last_value = train[col].iloc[-1]
                
                # Gentle trend continuation
                future_values = [last_value + trend * i * 0.5 for i in range(1, FORECAST_DAYS + 1)]
                future.loc[future['ds'] > train['ds'].max(), col] = future_values
                future.loc[future['ds'] <= train['ds'].max(), col] = train[col].values
        
        forecast = model.predict(future)
        
        # HYBRID: Add ARIMA for trend smoothing
        if USE_HYBRID:
            try:
                # Fit ARIMA on recent data
                recent_data = train['y'].tail(365).values  # Last year
                arima_model = fit_arima_trend(recent_data, order=(1, 1, 1))
                
                # Forecast with ARIMA
                arima_forecast = arima_model.forecast(steps=FORECAST_DAYS)
                
                # Get Prophet future predictions
                future_prophet = forecast[forecast['ds'] > train['ds'].max()]['yhat'].values
                
                # Blend: 70% Prophet + 30% ARIMA for smooth trend
                blended_forecast = 0.7 * future_prophet + 0.3 * arima_forecast
                
                # Apply to forecast
                forecast.loc[forecast['ds'] > train['ds'].max(), 'yhat'] = blended_forecast
                
                print("Hybrid Prophet + ARIMA applied")
            except Exception as e:
                print(f"ARIMA skipped: {e}")
        
        # --- Apply additional smoothing ---
        future_mask = forecast['ds'] > train['ds'].max()
        forecast.loc[future_mask, 'yhat'] = smooth_forecast(
            forecast.loc[future_mask, 'yhat'].values, 
            strength=SMOOTHING_STRENGTH
        )
        
        # Smooth confidence intervals too
        forecast.loc[future_mask, 'yhat_lower'] = smooth_forecast(
            forecast.loc[future_mask, 'yhat_lower'].values, 
            strength=SMOOTHING_STRENGTH * 0.8
        )
        forecast.loc[future_mask, 'yhat_upper'] = smooth_forecast(
            forecast.loc[future_mask, 'yhat_upper'].values, 
            strength=SMOOTHING_STRENGTH * 0.8
        )
        
        # Cross-validation
        try:
            total_days = (train['ds'].max() - train['ds'].min()).days
            
            if total_days >= 3650:
                cv_df = cross_validation(
                    model,
                    initial='2920 days',
                    period='182 days',
                    horizon='91 days',
                    parallel="processes"
                )
                
                mape = mean_absolute_percentage_error(cv_df['y'], cv_df['yhat']) * 100
                mae = mean_absolute_error(cv_df['y'], cv_df['yhat'])
                rmse = np.sqrt(mean_squared_error(cv_df['y'], cv_df['yhat']))
                accuracy = 100 - mape
                
                print(f"\n Cross-validation completed:")
                print(f"   Accuracy: {accuracy:.2f}%")
                print(f"   MAPE: {mape:.2f}%")
                print(f"   MAE: ${mae:.2f}")
                print(f"   RMSE: ${rmse:.2f}")
                
            else:
                mape, mae, rmse, accuracy = np.nan, np.nan, np.nan, np.nan
                
        except Exception as e:
            print(f" Cross-validation error: {e}")
            mape, mae, rmse, accuracy = np.nan, np.nan, np.nan, np.nan
        
        # --- Save Metrics ---
        metrics_summary.append({
            'Stock': stock_name,
            'Accuracy (%)': accuracy,
            'MAPE (%)': mape,
            'MAE ($)': mae,
            'RMSE ($)': rmse,
            'Avg_Price ($)': df['Close'].mean()
        })
        
        # --- Plot forecast ---
        forecast_start = train['ds'].iloc[-1]
        future_forecast = forecast[forecast['ds'] > forecast_start].copy()
        last_90_days = train.tail(90)
        
        if USE_INTERACTIVE_CHARTS:
            # Create interactive Plotly chart
            print("\n Generating interactive chart...")
            fig = create_interactive_chart(
                stock_name, 
                last_90_days, 
                future_forecast, 
                accuracy, 
                FORECAST_DAYS,
                train['y'].iloc[0]
            )
            fig.show()
        else:
            # Create static matplotlib chart
            create_static_chart(
                stock_name, 
                last_90_days, 
                future_forecast, 
                accuracy, 
                FORECAST_DAYS
            )
        
    except Exception as e:
        print(f" Error processing {stock_file}: {e}")
        continue

# --- Summary Results ---
metrics_df = pd.DataFrame(metrics_summary)
metrics_df = metrics_df.sort_values(by='Accuracy (%)', ascending=False).reset_index(drop=True)

print("\n" + "=" * 100)
print(" FINAL PERFORMANCE SUMMARY - ALL STOCKS (SMOOTH FORECASTS)")
print("=" * 100)
print(metrics_df.to_string(index=False))

high_accuracy_stocks = metrics_df[metrics_df['Accuracy (%)'] >= 85.0]

print("\n" + "=" * 100)
print(f" STOCKS WITH >= 85% ACCURACY: {len(high_accuracy_stocks)}/{len(metrics_df)}")
print("=" * 100)
if len(high_accuracy_stocks) > 0:
    print(high_accuracy_stocks.to_string(index=False))
else:
    print("No stocks reached 85% accuracy threshold yet.")

print("\n Average Accuracy Across All Stocks: {:.2f}%".format(metrics_df['Accuracy (%)'].mean()))
print(f" Median Accuracy: {metrics_df['Accuracy (%)'].median():.2f}%")
if len(metrics_df) > 0:
    print(f" Best Performing Stock: {metrics_df.iloc[0]['Stock']} ({metrics_df.iloc[0]['Accuracy (%)']:.2f}%)")
    
if SAVE_MODELS:
    print(f" Total models saved: {len(metrics_df)}")

 Training smooth forecast for: dataset/AMZN_stock_data.csv
Using data from 2015-10-12 to 2025-10-10
Total trading days: 2515


21:11:54 - cmdstanpy - INFO - Chain [1] start processing
21:11:56 - cmdstanpy - INFO - Chain [1] done processing


 Model saved: SaveModels\AMZN_model.pkl
Hybrid Prophet + ARIMA applied

 Cross-validation completed:
   Accuracy: 89.18%
   MAPE: 10.82%
   MAE: $20.28
   RMSE: $26.35

 Generating interactive chart...


 Training smooth forecast for: dataset/AAPL_stock_data.csv
Using data from 2015-10-12 to 2025-10-10
Total trading days: 2515


21:12:08 - cmdstanpy - INFO - Chain [1] start processing
21:12:09 - cmdstanpy - INFO - Chain [1] done processing


...

 Model saved: SaveModels\UNH_model.pkl
Hybrid Prophet + ARIMA applied

 Cross-validation completed:
   Accuracy: 97.87%
   MAPE: 2.13%
   MAE: $8.77
   RMSE: $11.27

 Generating interactive chart...


 Training smooth forecast for: dataset/V_stock_data.csv
Using data from 2015-10-12 to 2025-10-10
Total trading days: 2515


21:16:55 - cmdstanpy - INFO - Chain [1] start processing
21:16:57 - cmdstanpy - INFO - Chain [1] done processing


 Model saved: SaveModels\V_model.pkl
Hybrid Prophet + ARIMA applied

 Cross-validation completed:
   Accuracy: 99.23%
   MAPE: 0.77%
   MAE: $2.40
   RMSE: $3.35

 Generating interactive chart...



 FINAL PERFORMANCE SUMMARY - ALL STOCKS (SMOOTH FORECASTS)
Stock  Accuracy (%)  MAPE (%)   MAE ($)  RMSE ($)  Avg_Price ($)
  JNJ     99.350584  0.649416  1.053378  1.368412     145.892445
   PG     99.315524  0.684476  1.110973  1.467643     123.679920
    V     99.225654  0.774346  2.403157  3.348782     189.708489
  HON     99.157100  0.842900  1.741444  2.466989     173.140145
 CSCO     99.085900  0.914100  0.515890  0.705608      45.893060
  TRV     99.072746  0.927254  2.220633  3.034946     157.881903
   HD     99.070979  0.929021  3.506380  4.343479     256.923972
 MSFT     99.054987  0.945013  4.090766  5.259461     219.375153
  MCD     99.036273  0.963727  2.875601  3.888397     223.223773
  JPM     98.991989  1.008011  2.370394  3.314658     134.588887
  SHW     98.977583  1.022417  3.521038  4.389489     370.300648
  IBM     98.908195  1.091805  2.515085  3.564753     154.364408
  CVX     98.902952  1.097048  1.665817  2.191756     125.251507
   GS     98.797651  1.202349 